In [79]:
from tenacity import retry, stop_after_attempt 
import requests
from bs4 import BeautifulSoup
import pandas as pd 

df = pd.read_csv('suumo01.csv', index_col=0)

In [80]:
df.info

<bound method DataFrame.info of                       名称     カテゴリ             住所                最寄り駅   築年数  \
0           グランヴィスタ神田練塀町  賃貸マンション   東京都千代田区神田練塀町  東京メトロ日比谷線/秋葉原駅 歩4分   築5年   
1           グランヴィスタ神田練塀町  賃貸マンション   東京都千代田区神田練塀町   東京メトロ銀座線/末広町駅 歩5分   築5年   
2           グランヴィスタ神田練塀町  賃貸マンション   東京都千代田区神田練塀町      都営新宿線/岩本町駅 歩7分   築5年   
3            エルスタンザ神田須田町  賃貸マンション  東京都千代田区神田須田町１       ＪＲ山手線/神田駅 歩4分   築5年   
4            エルスタンザ神田須田町  賃貸マンション  東京都千代田区神田須田町１       ＪＲ山手線/神田駅 歩4分   築5年   
...                  ...      ...            ...                 ...   ...   
7060           ハミルトンプレイス  賃貸マンション     東京都千代田区三番町   東京メトロ東西線/九段下駅 歩9分  築18年   
7061           ハミルトンプレイス  賃貸マンション     東京都千代田区三番町  東京メトロ半蔵門線/半蔵門駅 歩9分  築18年   
7062  ＪＲ山手線 神田駅 4階建 築45年  賃貸マンション    東京都千代田区東神田１      ＪＲ山手線/神田駅 歩10分  築45年   
7063  ＪＲ山手線 神田駅 4階建 築45年  賃貸マンション    東京都千代田区東神田１     都営浅草線/東日本橋駅 歩5分  築45年   
7064  ＪＲ山手線 神田駅 4階建 築45年  賃貸マンション    東京都千代田区東神田１     都営新宿線/馬喰横山駅 歩2分  築45年   

             構造   階数      賃料   

In [81]:
df.drop_duplicates(subset=['階数','賃料','管理費','間取り','専有面積'], inplace=True)
df

,名称,カテゴリ,住所,最寄り駅,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,専有面積
0,グランヴィスタ神田練塀町,賃貸マンション,東京都千代田区神田練塀町,東京メトロ日比谷線/秋葉原駅 歩4分,築5年,地下2地上21階建,19階,58万円,-,116万円,-,3LDK,88.49m2
3,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,築5年,12階建,4階,14.4万円,10000円,14.4万円,-,1DK,29.02m2
4,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,築5年,12階建,8階,27.7万円,13000円,27.7万円,-,2LDK,54.42m2
5,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,築5年,12階建,11階,28.3万円,13000円,28.3万円,-,2LDK,54.42m2
12,KWレジデンス麹町,賃貸マンション,東京都千代田区麹町３,東京メトロ有楽町線/麹町駅 歩4分,築19年,地下1地上15階建,6階,16.2万円,15000円,16.2万円,16.2万円,1K,33.1m2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7041,コーポ麹町,賃貸マンション,東京都千代田区三番町,東京メトロ半蔵門線/半蔵門駅 歩9分,築54年,地下1地上9階建,4階,14.5万円,-,14.5万円,14.5万円,1DK,37.72m2
7044,東京メトロ半蔵門線 半蔵門駅 地下1地上9階建 築54年,賃貸マンション,東京都千代田区三番町,東京メトロ半蔵門線/半蔵門駅 歩9分,築54年,地下1地上9階建,4階,15万円,-,15万円,15万円,1DK,37.72m2
7047,ＪＲ山手線 秋葉原駅 5階建 築45年,賃貸マンション,東京都千代田区岩本町２,ＪＲ山手線/秋葉原駅 歩8分,築45年,5階建,5階,6.8万円,5000円,6.8万円,6.8万円,ワンルーム,15.47m2
7053,ＪＲ総武線 飯田橋駅 14階建 築11年,賃貸マンション,東京都千代田区神田神保町１,ＪＲ総武線/飯田橋駅 歩17分,築11年,14階建,7階,12.1万円,10000円,12.1万円,12.1万円,1K,25.06m2


In [82]:
df['最寄り駅_抽出'] = df['最寄り駅'].apply(lambda x: x.split('/')[0])
df

,名称,カテゴリ,住所,最寄り駅,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,専有面積,最寄り駅_抽出
0,グランヴィスタ神田練塀町,賃貸マンション,東京都千代田区神田練塀町,東京メトロ日比谷線/秋葉原駅 歩4分,築5年,地下2地上21階建,19階,58万円,-,116万円,-,3LDK,88.49m2,東京メトロ日比谷線
3,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,築5年,12階建,4階,14.4万円,10000円,14.4万円,-,1DK,29.02m2,ＪＲ山手線
4,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,築5年,12階建,8階,27.7万円,13000円,27.7万円,-,2LDK,54.42m2,ＪＲ山手線
5,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,築5年,12階建,11階,28.3万円,13000円,28.3万円,-,2LDK,54.42m2,ＪＲ山手線
12,KWレジデンス麹町,賃貸マンション,東京都千代田区麹町３,東京メトロ有楽町線/麹町駅 歩4分,築19年,地下1地上15階建,6階,16.2万円,15000円,16.2万円,16.2万円,1K,33.1m2,東京メトロ有楽町線
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7041,コーポ麹町,賃貸マンション,東京都千代田区三番町,東京メトロ半蔵門線/半蔵門駅 歩9分,築54年,地下1地上9階建,4階,14.5万円,-,14.5万円,14.5万円,1DK,37.72m2,東京メトロ半蔵門線
7044,東京メトロ半蔵門線 半蔵門駅 地下1地上9階建 築54年,賃貸マンション,東京都千代田区三番町,東京メトロ半蔵門線/半蔵門駅 歩9分,築54年,地下1地上9階建,4階,15万円,-,15万円,15万円,1DK,37.72m2,東京メトロ半蔵門線
7047,ＪＲ山手線 秋葉原駅 5階建 築45年,賃貸マンション,東京都千代田区岩本町２,ＪＲ山手線/秋葉原駅 歩8分,築45年,5階建,5階,6.8万円,5000円,6.8万円,6.8万円,ワンルーム,15.47m2,ＪＲ山手線
7053,ＪＲ総武線 飯田橋駅 14階建 築11年,賃貸マンション,東京都千代田区神田神保町１,ＪＲ総武線/飯田橋駅 歩17分,築11年,14階建,7階,12.1万円,10000円,12.1万円,12.1万円,1K,25.06m2,ＪＲ総武線


In [83]:
# 築年数の列を整数に変換する関数
def convert_age_to_int(age):
    if age == '新築':
        return 0
    else:
        return int(age.split('築')[1].replace('年',''))
    
df['築年数'] = df['築年数'].apply(convert_age_to_int)
df
    

,名称,カテゴリ,住所,最寄り駅,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,専有面積,最寄り駅_抽出
0,グランヴィスタ神田練塀町,賃貸マンション,東京都千代田区神田練塀町,東京メトロ日比谷線/秋葉原駅 歩4分,5,地下2地上21階建,19階,58万円,-,116万円,-,3LDK,88.49m2,東京メトロ日比谷線
3,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,5,12階建,4階,14.4万円,10000円,14.4万円,-,1DK,29.02m2,ＪＲ山手線
4,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,5,12階建,8階,27.7万円,13000円,27.7万円,-,2LDK,54.42m2,ＪＲ山手線
5,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,5,12階建,11階,28.3万円,13000円,28.3万円,-,2LDK,54.42m2,ＪＲ山手線
12,KWレジデンス麹町,賃貸マンション,東京都千代田区麹町３,東京メトロ有楽町線/麹町駅 歩4分,19,地下1地上15階建,6階,16.2万円,15000円,16.2万円,16.2万円,1K,33.1m2,東京メトロ有楽町線
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7041,コーポ麹町,賃貸マンション,東京都千代田区三番町,東京メトロ半蔵門線/半蔵門駅 歩9分,54,地下1地上9階建,4階,14.5万円,-,14.5万円,14.5万円,1DK,37.72m2,東京メトロ半蔵門線
7044,東京メトロ半蔵門線 半蔵門駅 地下1地上9階建 築54年,賃貸マンション,東京都千代田区三番町,東京メトロ半蔵門線/半蔵門駅 歩9分,54,地下1地上9階建,4階,15万円,-,15万円,15万円,1DK,37.72m2,東京メトロ半蔵門線
7047,ＪＲ山手線 秋葉原駅 5階建 築45年,賃貸マンション,東京都千代田区岩本町２,ＪＲ山手線/秋葉原駅 歩8分,45,5階建,5階,6.8万円,5000円,6.8万円,6.8万円,ワンルーム,15.47m2,ＪＲ山手線
7053,ＪＲ総武線 飯田橋駅 14階建 築11年,賃貸マンション,東京都千代田区神田神保町１,ＪＲ総武線/飯田橋駅 歩17分,11,14階建,7階,12.1万円,10000円,12.1万円,12.1万円,1K,25.06m2,ＪＲ総武線


In [84]:
import re

# 整数＋階数なら整数だけを抽出、それ以外は条件に従って置き換える関数
def process_floor(value):
    if re.match(r'\d+階', value):  # 整数＋階数の場合
        pattern = r'(\d+)'  # 正規表現で整数部分を抽出
        match = re.search(pattern, value)
        if match:
            return match.group(1)
    elif value == '-':  # '-' の場合
        return "1"
    else:  # 'B1-2階' のような場合
        return "1"

# 各要素を処理して新しい値を格納
df['階数'] = df['階数'].apply(process_floor)

In [85]:
# 賃料、管理費、敷金、礼金を整数に変換する関数
def convert_to_integer(value):
    if value == '-':
        return 0  # '-' がある場合は0に変換
    else:
        if '万円' in value:
            value = value.replace('万円', '')  # 「万円」を削除
            return int(float(value) * 10000)  # 万を桁に変換して整数に変換
        else:
            return int(value.replace('円', ''))  # 単位がない場合、円を削除して整数に変換

# 各列の値を変換
df['賃料'] = df['賃料'].apply(convert_to_integer)
df['管理費'] = df['管理費'].apply(convert_to_integer)
df['敷金'] = df['敷金'].apply(convert_to_integer)
df['礼金'] = df['礼金'].apply(convert_to_integer)

# 結果の表示
print(df)

                                名称     カテゴリ             住所  \
0                     グランヴィスタ神田練塀町  賃貸マンション   東京都千代田区神田練塀町   
3                      エルスタンザ神田須田町  賃貸マンション  東京都千代田区神田須田町１   
4                      エルスタンザ神田須田町  賃貸マンション  東京都千代田区神田須田町１   
5                      エルスタンザ神田須田町  賃貸マンション  東京都千代田区神田須田町１   
12                       KWレジデンス麹町  賃貸マンション     東京都千代田区麹町３   
...                            ...      ...            ...   
7041                         コーポ麹町  賃貸マンション     東京都千代田区三番町   
7044  東京メトロ半蔵門線 半蔵門駅 地下1地上9階建 築54年  賃貸マンション     東京都千代田区三番町   
7047           ＪＲ山手線 秋葉原駅 5階建 築45年  賃貸マンション    東京都千代田区岩本町２   
7053          ＪＲ総武線 飯田橋駅 14階建 築11年  賃貸マンション  東京都千代田区神田神保町１   
7062            ＪＲ山手線 神田駅 4階建 築45年  賃貸マンション    東京都千代田区東神田１   

                    最寄り駅  築年数         構造  階数      賃料    管理費       敷金      礼金  \
0     東京メトロ日比谷線/秋葉原駅 歩4分    5  地下2地上21階建  19  580000      0  1160000       0   
3          ＪＲ山手線/神田駅 歩4分    5       12階建   4  144000  10000   144000       0   
4          ＪＲ山手

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1341 entries, 0 to 7062
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   名称       1341 non-null   object
 1   カテゴリ     1341 non-null   object
 2   住所       1341 non-null   object
 3   最寄り駅     1341 non-null   object
 4   築年数      1341 non-null   int64 
 5   構造       1341 non-null   object
 6   階数       1341 non-null   object
 7   賃料       1341 non-null   int64 
 8   管理費      1341 non-null   int64 
 9   敷金       1341 non-null   int64 
 10  礼金       1341 non-null   int64 
 11  間取り      1341 non-null   object
 12  専有面積     1341 non-null   object
 13  最寄り駅_抽出  1341 non-null   object
dtypes: int64(5), object(9)
memory usage: 157.1+ KB


In [87]:
# m2を削除する関数
def remove_m2(value):
    return float(value.replace('m2', ''))

# 面積の単位である 'm2' を削除
df['専有面積'] = df['専有面積'].apply(remove_m2)

# 結果の表示
print(df)

                                名称     カテゴリ             住所  \
0                     グランヴィスタ神田練塀町  賃貸マンション   東京都千代田区神田練塀町   
3                      エルスタンザ神田須田町  賃貸マンション  東京都千代田区神田須田町１   
4                      エルスタンザ神田須田町  賃貸マンション  東京都千代田区神田須田町１   
5                      エルスタンザ神田須田町  賃貸マンション  東京都千代田区神田須田町１   
12                       KWレジデンス麹町  賃貸マンション     東京都千代田区麹町３   
...                            ...      ...            ...   
7041                         コーポ麹町  賃貸マンション     東京都千代田区三番町   
7044  東京メトロ半蔵門線 半蔵門駅 地下1地上9階建 築54年  賃貸マンション     東京都千代田区三番町   
7047           ＪＲ山手線 秋葉原駅 5階建 築45年  賃貸マンション    東京都千代田区岩本町２   
7053          ＪＲ総武線 飯田橋駅 14階建 築11年  賃貸マンション  東京都千代田区神田神保町１   
7062            ＪＲ山手線 神田駅 4階建 築45年  賃貸マンション    東京都千代田区東神田１   

                    最寄り駅  築年数         構造  階数      賃料    管理費       敷金      礼金  \
0     東京メトロ日比谷線/秋葉原駅 歩4分    5  地下2地上21階建  19  580000      0  1160000       0   
3          ＪＲ山手線/神田駅 歩4分    5       12階建   4  144000  10000   144000       0   
4          ＪＲ山手

In [88]:
# "歩x分" を抽出する関数
def extract_walk_time(text):
    match = re.search(r'歩\d+分', text)
    if match:
        return match.group()

# "歩x分" を抽出して新しい列に格納
df['歩行時間'] = df['最寄り駅'].apply(extract_walk_time)

# 結果の表示
print(df)

                                名称     カテゴリ             住所  \
0                     グランヴィスタ神田練塀町  賃貸マンション   東京都千代田区神田練塀町   
3                      エルスタンザ神田須田町  賃貸マンション  東京都千代田区神田須田町１   
4                      エルスタンザ神田須田町  賃貸マンション  東京都千代田区神田須田町１   
5                      エルスタンザ神田須田町  賃貸マンション  東京都千代田区神田須田町１   
12                       KWレジデンス麹町  賃貸マンション     東京都千代田区麹町３   
...                            ...      ...            ...   
7041                         コーポ麹町  賃貸マンション     東京都千代田区三番町   
7044  東京メトロ半蔵門線 半蔵門駅 地下1地上9階建 築54年  賃貸マンション     東京都千代田区三番町   
7047           ＪＲ山手線 秋葉原駅 5階建 築45年  賃貸マンション    東京都千代田区岩本町２   
7053          ＪＲ総武線 飯田橋駅 14階建 築11年  賃貸マンション  東京都千代田区神田神保町１   
7062            ＪＲ山手線 神田駅 4階建 築45年  賃貸マンション    東京都千代田区東神田１   

                    最寄り駅  築年数         構造  階数      賃料    管理費       敷金      礼金  \
0     東京メトロ日比谷線/秋葉原駅 歩4分    5  地下2地上21階建  19  580000      0  1160000       0   
3          ＪＲ山手線/神田駅 歩4分    5       12階建   4  144000  10000   144000       0   
4          ＪＲ山手

In [89]:
df

,名称,カテゴリ,住所,最寄り駅,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,専有面積,最寄り駅_抽出,歩行時間
0,グランヴィスタ神田練塀町,賃貸マンション,東京都千代田区神田練塀町,東京メトロ日比谷線/秋葉原駅 歩4分,5,地下2地上21階建,19,580000,0,1160000,0,3LDK,88.49,東京メトロ日比谷線,歩4分
3,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,5,12階建,4,144000,10000,144000,0,1DK,29.02,ＪＲ山手線,歩4分
4,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,5,12階建,8,277000,13000,277000,0,2LDK,54.42,ＪＲ山手線,歩4分
5,エルスタンザ神田須田町,賃貸マンション,東京都千代田区神田須田町１,ＪＲ山手線/神田駅 歩4分,5,12階建,11,283000,13000,283000,0,2LDK,54.42,ＪＲ山手線,歩4分
12,KWレジデンス麹町,賃貸マンション,東京都千代田区麹町３,東京メトロ有楽町線/麹町駅 歩4分,19,地下1地上15階建,6,162000,15000,162000,162000,1K,33.10,東京メトロ有楽町線,歩4分
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7041,コーポ麹町,賃貸マンション,東京都千代田区三番町,東京メトロ半蔵門線/半蔵門駅 歩9分,54,地下1地上9階建,4,145000,0,145000,145000,1DK,37.72,東京メトロ半蔵門線,歩9分
7044,東京メトロ半蔵門線 半蔵門駅 地下1地上9階建 築54年,賃貸マンション,東京都千代田区三番町,東京メトロ半蔵門線/半蔵門駅 歩9分,54,地下1地上9階建,4,150000,0,150000,150000,1DK,37.72,東京メトロ半蔵門線,歩9分
7047,ＪＲ山手線 秋葉原駅 5階建 築45年,賃貸マンション,東京都千代田区岩本町２,ＪＲ山手線/秋葉原駅 歩8分,45,5階建,5,68000,5000,68000,68000,ワンルーム,15.47,ＪＲ山手線,歩8分
7053,ＪＲ総武線 飯田橋駅 14階建 築11年,賃貸マンション,東京都千代田区神田神保町１,ＪＲ総武線/飯田橋駅 歩17分,11,14階建,7,121000,10000,121000,121000,1K,25.06,ＪＲ総武線,歩17分


In [90]:
df.to_csv("suumo02_2.csv", encoding="utf-8")